In [1]:
from spmf import Spmf
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ast
pd.options.plotting.backend = "plotly"
import plotly.express as px
# from chart_studio import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn.feature_extraction.text import CountVectorizer
pd.options.display.max_colwidth = None
pd.set_option('display.max_rows', 70)
from pandas.plotting import table 
#import nltk

import nltk
import seaborn as sns
%matplotlib inline
import random
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import NMF
import plotly.io as pio
import json
import pickle

In [2]:
col_list1 = ["user_id", "package_name",'session','user_median_session_length']
data_length1_session = pd.read_csv("data_length1_session_grouped.csv", usecols=col_list1)

In [3]:
data_length1_session['group']='group1'

In [4]:
col_list = ["user_id", "package_name",'session','user_median_session_length']
data_length2_session = pd.read_csv("data_length2_session_grouped.csv", usecols=col_list)

In [5]:
data_length2_session['group']='group2'

In [6]:
col_list2 = ["user_id", "package_name",'session','user_median_session_length']
data_length_other_session = pd.read_csv("data_length_other_session_grouped.csv", usecols=col_list2)

In [7]:
data_length_other_session['group']='group3'

In [15]:
data_length_other_session

,user_id,package_name,session,user_median_session_length,group
0,8,com.google.android.googlequicksearchbox,0,3.0,group3
1,8,com.android.chrome,0,3.0,group3
2,8,com.google.android.googlequicksearchbox,0,3.0,group3
3,8,com.netmarble.sknightsgb,0,3.0,group3
4,8,com.instagram.android,0,3.0,group3
...,...,...,...,...,...
25166108,9994,com.twitter.android,2919,3.0,group3
25166109,9994,com.nutsplay.slg2.en.ccgg,2920,3.0,group3
25166110,9994,com.facebook.katana,2920,3.0,group3
25166111,9994,com.sonymobile.email,2921,3.0,group3


In [16]:
full_data=pd.concat([data_length1_session,data_length2_session,data_length_other_session],axis=0)

In [17]:
full_data

,user_id,package_name,session,user_median_session_length,group
0,1,com.sonymobile.moviecreator.rmm,0,1.0,group1
1,1,com.yingwen.photographertoolspro,1,1.0,group1
2,1,net.osmand.plus,1,1.0,group1
3,1,com.android.phone,1,1.0,group1
4,1,com.sonyericsson.album,2,1.0,group1
...,...,...,...,...,...
25166108,9994,com.twitter.android,2919,3.0,group3
25166109,9994,com.nutsplay.slg2.en.ccgg,2920,3.0,group3
25166110,9994,com.facebook.katana,2920,3.0,group3
25166111,9994,com.sonymobile.email,2921,3.0,group3


In [19]:
full_data.to_csv('full_data_session_length',index=False)

In [2]:
full_data = pd.read_csv('full_data_session_length.csv')

In [3]:
full_data

,user_id,package_name,session,user_median_session_length,group
0,1,com.sonymobile.moviecreator.rmm,0,1.0,group1
1,1,com.yingwen.photographertoolspro,1,1.0,group1
2,1,net.osmand.plus,1,1.0,group1
3,1,com.android.phone,1,1.0,group1
4,1,com.sonyericsson.album,2,1.0,group1
...,...,...,...,...,...
228733591,9994,com.twitter.android,2919,3.0,group3
228733592,9994,com.nutsplay.slg2.en.ccgg,2920,3.0,group3
228733593,9994,com.facebook.katana,2920,3.0,group3
228733594,9994,com.sonymobile.email,2921,3.0,group3


# N-gram of session length 1 group 

In [8]:
data_length1_session['start']=data_length1_session.groupby('user_id').session.shift(1)!=data_length1_session.session

In [9]:
data_length1_session['end']=data_length1_session.groupby('user_id').session.shift(-1)!=data_length1_session.session

In [10]:
user_corpus_length1_session=data_length1_session.groupby(['user_id','session']).package_name.apply(' '.join)

In [11]:
user_corpus_length1_session_with_start_end='start '+user_corpus_length1_session+ ' end'

In [12]:
print("Number of sessions:",user_corpus_length1_session.shape[0])
print("0.75% of sessions:",user_corpus_length1_session.shape[0]/100*0.75)

Number of sessions: 11689790
0.75% of sessions: 87673.42499999999


In [13]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)
vectorization=vectorizer.fit_transform(user_corpus_length1_session_with_start_end.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]

array(['bbc.mobile.news.uk', 'com.android.chrome', 'com.android.email',
       'com.android.incallui', 'com.ebay.mobile', 'com.facebook.katana',
       'com.facebook.orca', 'com.google.android.apps.maps',
       'com.google.android.apps.messaging', 'com.google.android.gm',
       'com.google.android.googlequicksearchbox',
       'com.google.android.youtube', 'com.instagram.android',
       'com.microsoft.office.outlook', 'com.nianticlabs.pokemongo',
       'com.snapchat.android', 'com.sony.nfx.app.sfrc',
       'com.sonyericsson.album',
       'com.sonyericsson.android.socialphonebook',
       'com.sonyericsson.conversations', 'com.sonyericsson.music',
       'com.sonymobile.calendar', 'com.sonymobile.email',
       'com.spotify.music', 'com.twitter.android', 'com.whatsapp',
       'com.yahoo.mobile.client.android.mail', 'end', 'start'],
      dtype=object)

In [14]:
frequent_apps_length1_grouped=vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]

In [11]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]))

Number of frequent patterns: 29


In [12]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))
vectorization=vectorizer.fit_transform(user_corpus_length1_session.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]

array(['com.android.chrome com.facebook.katana',
       'com.android.chrome com.google.android.googlequicksearchbox',
       'com.android.chrome com.whatsapp',
       'com.android.incallui com.sonyericsson.android.socialphonebook',
       'com.facebook.katana com.android.chrome',
       'com.facebook.katana com.facebook.orca',
       'com.facebook.katana com.sonyericsson.conversations',
       'com.facebook.katana com.whatsapp',
       'com.facebook.orca com.facebook.katana',
       'com.google.android.googlequicksearchbox com.android.chrome',
       'com.sonyericsson.android.socialphonebook com.android.incallui',
       'com.sonyericsson.conversations com.facebook.katana',
       'com.sonyericsson.conversations com.whatsapp',
       'com.whatsapp com.android.chrome',
       'com.whatsapp com.facebook.katana',
       'com.whatsapp com.sonyericsson.conversations'], dtype=object)

In [13]:
len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]])

16

In [14]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization=vectorizer.fit_transform(user_corpus_length1_session.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]

array(['com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox'],
      dtype=object)

In [15]:
len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]])

1

In [16]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4))
vectorization=vectorizer.fit_transform(user_corpus_length1_session.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]]

array([], dtype=object)

In [17]:
len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=87673)[1]])

0

# N-gram of  session length 2 group

In [15]:
data_length2_session['start']=data_length2_session.groupby('user_id').session.shift(1)!=data_length2_session.session

In [16]:
data_length2_session['end']=data_length2_session.groupby('user_id').session.shift(-1)!=data_length2_session.session

In [17]:
user_corpus_length2_session=data_length2_session.groupby(['user_id','session']).package_name.apply(' '.join)

In [18]:
user_corpus_with_start_end_length2_session='start '+user_corpus_length2_session+ ' end'

In [19]:
print("Number of sessions:",user_corpus_with_start_end_length2_session.shape[0])
print("0.75% of sessions:",user_corpus_with_start_end_length2_session.shape[0]/100*0.75)

Number of sessions: 60142439
0.75% of sessions: 451068.2925


In [20]:
vectorizer_1gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)

In [21]:
vectorization_1gram=vectorizer_1gram.fit_transform(user_corpus_with_start_end_length2_session.values)

In [22]:
frequent_apps_length2_grouped=vectorizer_1gram.get_feature_names_out()[np.where((vectorization_1gram>0).sum(axis=0)>150356)[1]]
print("Number frequent apps:",len(frequent_apps_length2_grouped))#75
frequent_apps_length2_grouped

Number frequent apps: 75


array(['bbc.mobile.news.uk', 'bbc.mobile.weather', 'com.amazon.kindle',
       'com.amazon.mShop.android.shopping', 'com.amazon.mp3',
       'com.android.chrome', 'com.android.email', 'com.android.incallui',
       'com.android.phone', 'com.barclays.android.barclaysmobilebanking',
       'com.bskyb.skynews.android', 'com.bskyb.sportnews',
       'com.dailymail.online', 'com.ebay.mobile', 'com.facebook.katana',
       'com.facebook.orca', 'com.fitbit.FitbitMobile',
       'com.google.android.apps.docs', 'com.google.android.apps.inbox',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging',
       'com.google.android.apps.photos', 'com.google.android.calendar',
       'com.google.android.gm', 'com.google.android.gms',
       'com.google.android.googlequicksearchbox',
       'com.google.android.keep', 'com.google.android.music',
       'com.google.android.talk', 'com.google.android.youtube',
       'com.grppl.android.shell.CMBlloydsTSB73',
       'com.grppl.and

In [ ]:
vectorizer_2gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))
vectorization_2gram=vectorizer_2gram.fit_transform(user_corpus_with_start_end_length2_session.values)
print("Number of 2-grams:",len(vectorizer_2gram.get_feature_names_out()[np.where((vectorization_2gram>0).sum(axis=0)>150356)[1]]))#134

In [ ]:
length_2_patterns = vectorizer_2gram.get_feature_names_out()[np.where((vectorization_2gram>0).sum(axis=0)>150356)[1]]
length_2_patterns

In [61]:
vectorizer_3gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization_3gram=vectorizer_3gram.fit_transform(user_corpus_length2_session.values)
print("Number of 3-grams:",len(vectorizer_3gram.get_feature_names_out()[np.where((vectorization_3gram>0).sum(axis=0)>104110)[1]]))#30
vectorizer_3gram.get_feature_names_out()[np.where((vectorization_3gram>0).sum(axis=0)>150356)[1]]


Number of 3-grams: 33


array(['com.android.chrome com.facebook.katana com.android.chrome',
       'com.android.chrome com.google.android.googlequicksearchbox com.android.chrome',
       'com.android.chrome com.whatsapp com.android.chrome',
       'com.facebook.katana com.android.chrome com.facebook.katana',
       'com.facebook.katana com.facebook.orca com.facebook.katana',
       'com.facebook.katana com.google.android.googlequicksearchbox com.android.chrome',
       'com.facebook.katana com.sonyericsson.conversations com.facebook.katana',
       'com.facebook.katana com.whatsapp com.facebook.katana',
       'com.facebook.orca com.facebook.katana com.facebook.orca',
       'com.google.android.googlequicksearchbox com.android.chrome com.facebook.katana',
       'com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox',
       'com.google.android.googlequicksearchbox com.android.chrome com.whatsapp',
       'com.sonyericsson.android.socialphonebook com.android.incall

In [62]:
vectorizer_4gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4))
vectorization_4gram=vectorizer_4gram.fit_transform(user_corpus_length2_session.values)
print("Number of 4-grams:",len(vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>104110)[1]]))
#vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>104110)[1]]
vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>150356)[1]]#4


Number of 4-grams: 7


array(['com.facebook.katana com.whatsapp com.facebook.katana com.whatsapp',
       'com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox com.android.chrome',
       'com.whatsapp com.facebook.katana com.whatsapp com.facebook.katana'],
      dtype=object)

# N-gram for length above 2 session group

In [13]:
data_length_other_session = pd.read_csv('data_length_other_session_grouped.csv')

In [23]:
data_length_other_session['start']=data_length_other_session.groupby('user_id').session.shift(1)!=data_length_other_session.session

In [24]:
data_length_other_session['end']=data_length_other_session.groupby('user_id').session.shift(-1)!=data_length_other_session.session

In [25]:
user_corpus_last_group=data_length_other_session.groupby(['user_id','session']).package_name.apply(' '.join)

In [26]:
user_corpus_with_start_end_last_group='start '+user_corpus_last_group+ ' end'

In [27]:
print("Number of sessions:",user_corpus_with_start_end_last_group.shape[0])
print("0.75% of sessions:",user_corpus_with_start_end_last_group.shape[0]/100*0.75)

Number of sessions: 4932851
0.75% of sessions: 36996.3825


In [28]:
vectorizer_1gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1),lowercase=False)

In [29]:
vectorization_1gram=vectorizer_1gram.fit_transform(user_corpus_with_start_end_last_group.values)

In [30]:
frequent_apps_other_group=vectorizer_1gram.get_feature_names_out()[np.where((vectorization_1gram>0).sum(axis=0)>12332)[1]]
print("Number frequent apps:",len(frequent_apps_other_group))#77
frequent_apps_other_group

Number frequent apps: 93


array(['bbc.mobile.news.uk', 'bbc.mobile.weather',
       'co.uk.Nationwide.Mobile', 'com.accuweather.android',
       'com.amazon.mShop.android.shopping', 'com.amazon.mp3',
       'com.android.browser', 'com.android.chrome', 'com.android.email',
       'com.android.incallui', 'com.android.phone', 'com.badoo.mobile',
       'com.barclays.android.barclaysmobilebanking', 'com.bskyb.fbscore',
       'com.bskyb.skynews.android', 'com.bskyb.sportnews',
       'com.contextlogic.wish', 'com.dailymail.online', 'com.discord',
       'com.ebay.mobile', 'com.facebook.katana', 'com.facebook.orca',
       'com.fitbit.FitbitMobile', 'com.google.android.apps.docs',
       'com.google.android.apps.genie.geniewidget',
       'com.google.android.apps.inbox',
       'com.google.android.apps.magazines',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging',
       'com.google.android.apps.photos', 'com.google.android.apps.plus',
       'com.google.android.calendar', 'com.google

In [91]:
vectorizer_2gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))
vectorization_2gram=vectorizer_2gram.fit_transform(user_corpus_last_group.values)
print("Number of 2-grams:",len(vectorizer_2gram.get_feature_names_out()[np.where((vectorization_2gram>0).sum(axis=0)>12332)[1]]))#134

Number of 2-grams: 176


In [92]:
vectorizer_3gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization_3gram=vectorizer_3gram.fit_transform(user_corpus_last_group.values)
print("Number of 3-grams:",len(vectorizer_3gram.get_feature_names_out()[np.where((vectorization_3gram>0).sum(axis=0)>12332)[1]]))#30
length_3_patterns = vectorizer_3gram.get_feature_names_out()[np.where((vectorization_3gram>0).sum(axis=0)>12332)[1]]
length_3_patterns

Number of 3-grams: 91


array(['com.android.chrome com.facebook.katana com.android.chrome',
       'com.android.chrome com.facebook.katana com.facebook.orca',
       'com.android.chrome com.facebook.katana com.whatsapp',
       'com.android.chrome com.facebook.orca com.facebook.katana',
       'com.android.chrome com.google.android.googlequicksearchbox com.android.chrome',
       'com.android.chrome com.google.android.googlequicksearchbox com.facebook.katana',
       'com.android.chrome com.whatsapp com.android.chrome',
       'com.android.chrome com.whatsapp com.facebook.katana',
       'com.facebook.katana com.android.chrome com.facebook.katana',
       'com.facebook.katana com.facebook.orca com.android.chrome',
       'com.facebook.katana com.facebook.orca com.facebook.katana',
       'com.facebook.katana com.facebook.orca com.instagram.android',
       'com.facebook.katana com.facebook.orca com.snapchat.android',
       'com.facebook.katana com.facebook.orca com.whatsapp',
       'com.facebook.katana com.

In [93]:
vectorizer_4gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4))
vectorization_4gram=vectorizer_4gram.fit_transform(user_corpus_last_group.values)
print("Number of 4-grams:",len(vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>12332)[1]]))
#vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>104110)[1]]
length_4_patterns = vectorizer_4gram.get_feature_names_out()[np.where((vectorization_4gram>0).sum(axis=0)>12332)[1]]#4
length_4_patterns

Number of 4-grams: 10


array(['com.android.chrome com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox',
       'com.facebook.katana com.facebook.orca com.facebook.katana com.facebook.orca',
       'com.facebook.katana com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox',
       'com.facebook.katana com.whatsapp com.facebook.katana com.whatsapp',
       'com.facebook.orca com.facebook.katana com.facebook.orca com.facebook.katana',
       'com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox com.android.chrome',
       'com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox com.facebook.katana',
       'com.instagram.android com.snapchat.android com.instagram.android com.snapchat.android',
       'com.snapchat.android com.instagram.android com.snapchat.android com.instagram.android',
       'com.whatsapp com.facebook.katana com.w

In [96]:
vectorizer_5gram = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5))
vectorization_5gram=vectorizer_5gram.fit_transform(user_corpus_last_group.values)
print("Number of 5-grams:",len(vectorizer_5gram.get_feature_names_out()[np.where((vectorization_5gram>0).sum(axis=0)>12332)[1]]))
#vectorizer_5gram.get_feature_names_out()[np.where((vectorization_5gram>0).sum(axis=0)>191938)[1]]
length_5_patterns = vectorizer_5gram.get_feature_names_out()[np.where((vectorization_5gram>0).sum(axis=0)>12332)[1]]
length_5_patterns

Number of 5-grams: 0


array([], dtype=object)

## Generate full frequnet apps

In [ ]:
freuqent_apps = set(frequent_apps_length1_grouped).union(set(frequent_apps_length2_grouped),set(frequent_apps_other_group))

In [32]:
with open('frequent_apps_session_length.txt','wb') as f:
    pickle.dump(freuqent_apps, f)

##To read it again from file

In [6]:
with open('frequent_apps_session_length.txt','rb') as f:
    freuqent_apps = pickle.load(f)